In [ ]:
#======================================================================================================
# by kobongsoo, 
# ============= HISTORY =============================================
# make by: 2022-04-13
# updata: 2022-09-27
# => KlueNLI 데이터 셋 추가
# => 데이터 셋은 KorNLI(550,152) 가 KlueNLI(24,998)보다 20배정도 크지만, 
# Klue 데이터 셋이 실제 모델 훈련에는 더 좋다고 함.
# ======================================================================
#
# sentence-bert NLI 훈련 및 평가 예시
# => 기존 (distil)bert 모델을 가지고, NLI로 훈련 및 평가 후, S-BERT로 만드는 예시임.
#
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄일수 있음
#=> reduce_out_dimension = True 로 하면, 출력 임베딩 dimension이 줄어들게 설정가능함
#
# => sentence-transformers 패키지를 이용하여 구현 함.(*pip install -U sentence-transformers 설치 필요)
#
# 도큐먼트 : https://www.sbert.net/index.html
# 소스참고 : https://github.com/BM-K/KoSentenceBERT-ETRI

# pip install -U sentence-transformers
#======================================================================================================
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../log/s-bert")
device = GPU_info()

In [ ]:
#s-bert로 만들 원본 bert 경로
#model_path = "bongsoo/albert-small-kor-sbert-v1.1" 
model_path = '../../data11/model/moco/kpf-max/kpf-max-sts-distil-nli-sts'

#원본 bert를 sentencebert로 만든후 만들어진 s-bert 저장 경로
#=> **해당 경로\eval 폴더에 similarity_evaluation_sts-dev_result.csv 파일로 각 epoch 마다 평가된 결과가 기록된다.
#smodel_path = 'output/training_nli_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S") 
smodel_path = "../../data11/model/moco/kpf-max/kpf-max-sts-distil-nli-sts-nli"

# 임베딩 벡터 폴링 모드 선택 (*아래값중 문자열로 입력함, 기본=mean)
# mean=단어 평균, max=최대값, cls=문장, 
#['mean', 'max', 'cls', 'weightedmean', 'lasttoken']
pooling_mode = 'max'

# 유사도 측정방식(COSINE, EUCLIDEAN, MANHATTAN, DOT_PRODUCT 중 선택 , 모두 spearman 방식임)
# => None 이면 아래 값들중 MAX 값 추력함
#main_similarity = None
main_similarity = SimilarityFunction.COSINE
#main_similarity = SimilarityFunction.EUCLIDEAN
#main_similarity = SimilarityFunction.MANHATTAN
#main_similarity = SimilarityFunction.DOT_PRODUCT

use_kornli = 1 # kornli 파일 
use_kluenli = 1 # kluests_v1.1 파일 
use_gluenli = 1 # glue 파일

#KorNLI, KorSTS 파일 경로
train_kornli_file = '../../data11/korpora/kornli/snli_1.0_train.ko.tsv' 
eval_korsts_file = '../../data11/korpora/korsts/tune_dev.tsv'

#KLUENIL, KlueSTS 파일 경로
train_kluenli_file = '../../data11/korpora/klue-nli/klue-nli-v1.1_train.json' 
eval_kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

#GLUENLI, GLUESTS 파일 경로
train_gluenli_file = '../../data11/korpora/gluemnli/glue-mnli-train.tsv' 
eval_gluests_file = '../../data11/korpora/gluestsb/gluestsb-valid.tsv'

#============================================================================ 
train_batch_size = 64 # 64 혹은 32 가 좋음 
eval_batch_size = 64 
num_epochs = 3 # 3 정도면 최상의 값 찾을수 있음 
max_seq_length = 128 
lr = 3e-5 # default=2e-5 
eps = 1e-8 #lr이 0으로 나뉘어져 계산이 엉키는 것을 방지하기 위해 epsilion
seed = 111 
#============================================================================
#* True=모델에 상관없이 영어는 소문자로 입력하겠다는 뜻(glue는 소문자로만 비교하는게 더 적확함), 한국어는 True도 상관없음
do_lower_case = True 
#do_lower_case = False
#============================================================================
#*출력 dimension을 줄일 경우에는 True로 하고, out_dimension에 줄일 값을 설정함
reduce_out_dimension = False # True이면 dimension을 줄임=>Dense 모델 추가됨 
out_dimension = 128 
#============================================================================

seed_everything(seed)

#모델과 tokenizer 를 불러옴
#=> **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

#word embedding_model 출력
print(word_embedding_model)

In [ ]:
# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode=pooling_mode)  # 워드 임베딩 값중 max 값을 이용
# pooling model 출력 
print(pooling_model)
print(pooling_model.get_sentence_embedding_dimension())

In [ ]:
# 3. dense 모델 추가(옵션)
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄임.
#=> https://www.sbert.net/docs/training/overview.html?highlight=dense 참조
if reduce_out_dimension:
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), # 입력 dimension은 앞에 pooling모델 embedding dimension으로 지정
                               out_features=out_dimension,  # 출력 dimension
                               activation_function=nn.Tanh())  # activation function은 Tahn으로 정의

In [ ]:
# SBERT 모델 생성
if reduce_out_dimension:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
else:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
print(model)

In [ ]:
# 훈련 데이터 불러오기
# => [sentence1, sentence2], labels 식으로 만듬
label2int = {"entailment": 0, "neutral": 1, "contradiction": 2}
train_samples = []

####################################################################################################
# KorNLI 훈련 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_kornli == 1:
    count = 0
    logger.info(f"\r\nRead NLI train dataset:{train_kornli_file}")

    with open(train_kornli_file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            s1, s2, label = line.split('\t')
            label = label2int[label.strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")
            
            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
        
    logger.info(f'*kornli len: {count}')
####################################################################################################

####################################################################################################
# KlueNLI 훈련 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "nli", split="train")
# for data in datas:
#        s1 = data["sentence1"]
#        s2 = data["sentence2"]
#        label = data["label"]["label"]
###################################################################################################    
# kluenli 훈련인 경우 
if use_kluenli == 1:
    count = 0
    import json
    logger.info(f"\r\nRead NLI train dataset:{train_kluenli_file}")

    with open(train_kluenli_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            #print(data)
            s1 = data["premise"].strip()
            s2 = data["hypothesis"].strip()
            label = label2int[data["gold_label"].strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")

            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
            
    logger.info(f'*kluenli len: {count}')
    
####################################################################################################
# GLUENLI 훈련 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_gluenli == 1:
    count = 0
    logger.info(f"\r\nRead NLI train dataset:{train_gluenli_file}")

    with open(train_gluenli_file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            s1, s2, label = line.split('\t')
            label = label2int[label.strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")
            
            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
        
    logger.info(f'*gluenli len: {count}')
####################################################################################################
        
logger.info(f'*train_samples_len:{len(train_samples)}')

In [ ]:
# 데이터 셋, 데이터 로더, 손실함수 정의
train_dataset = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

# train_loss 정의 
# => 기본은 MSELoss 인데, NLI 훈련시에는 MultilpleNegativesRankingLoss(MNR Loss)가 좋다고 함.
train_loss = losses.MultipleNegativesRankingLoss(model)
#train_loss = losses.SoftmaxLoss(model=model, 
#                                sentence_embedding_dimension=model.get_sentence_embedding_dimension(), 
#                                num_labels=len(label2int))

In [ ]:
from datasets import load_dataset

#Read STSbenchmark dataset and use it as development set
# 평가데이터 불러오기
#korsts 파일로 두 문장간 유사도를 수치로(5.0이 만점=매우 유사) 측정함.
dev_samples = []

####################################################################################################
# KorSTS 평가 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_kornli == True:
    count = 0
    logger.info(f"\r\nRead STS dev dataset=>{eval_korsts_file}")
    with open(eval_korsts_file, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 5:
                print(f"{text_a}, {text_b}, {score}")
            
            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
        logger.info(f'*korsts len: {count}')
####################################################################################################  

####################################################################################################
# KlueSTS 평가 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "sts", split="test")
# for data in datas:
#        text_a = data["sentence1"]
#        text_b = data["sentence2"]
#        score = data["labels"]["label"]
#        score = float(score) / 5.0  
####################################################################################################           
if use_kluenli == True:
    count = 0
    logger.info(f"\r\nRead STS dev dataset=>{eval_kluests_file}")
    with open(eval_kluests_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            text_a = data["sentence1"]
            text_b = data["sentence2"]
            score = data["labels"]["label"]
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

            if count < 5:
                print(f"{text_a}, {text_b}, {score}")

            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
        logger.info(f'*kluests len: {count}')
####################################################################################################  

####################################################################################################
# GLUESTS 평가 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_gluenli == True:
    count = 0
    #logger.info(f"\r\nRead STS dev dataset=>{eval_gluests_file}")
    
    #glue stsb valildation 데이터셋 불러옴(subset : "stsb" = 1,500)
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        if count < 5:
            print(f"{text_a}, {text_b}, {score}")
                
        dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
        
    '''
    with open(eval_gluests_file, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 5:
                print(f"{text_a}, {text_b}, {score}")
            
            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
    '''
    logger.info(f'*gluests len: {count}')
####################################################################################################  
    
# 2개의 bert 모델에서 구한 2개의 embedding 값들의 cosine 유사도를 구해서, 이를 실제 score와 비교해서 유사도 측정함
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, 
                                                                 main_similarity=main_similarity,
                                                                 batch_size=eval_batch_size, 
                                                                 name='sts-dev')

In [ ]:

warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up

# evaluation_steps은 20%로 설정
evaluation_steps = int(len(train_dataset) * num_epochs / eval_batch_size * 0.2)
logger.info(f"model:{model_path}, smodel:{smodel_path}")
logger.info("*train_batch: {}, eval_batch:{}, epoch:{}, lr:{}, eps:{}, train_dataset:{}, Warmup-steps: {}, evaluation_step: {}".format(train_batch_size, eval_batch_size, num_epochs, lr, eps, len(train_dataset), warmup_steps, evaluation_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          optimizer_params= {'lr': lr, 'eps': eps, 'correct_bias': False},
          save_best_model=True, # **기본 = True : eval 가장 best 모델을 output_Path에 저장함
          output_path=smodel_path
          )


In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
# => 훈련되어서 저장된 s-bert 모델을 불러와서 성능 평가 해봄
##############################################################################
import time

# 훈련완료후 테스트해볼 파일 경로(*Kluests_test 파일은 없어서, korsts test 파일 이용)
test_sts = '../../data11/korpora/korsts/tune_test.tsv'
test_batch_size = 32

# 테스트시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
test_output_path = "./output"
os.makedirs(test_output_path, exist_ok=True)

test_samples = []
with open(test_sts, 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        test_samples.append(InputExample(texts=[s1,s2], label=score))

logger.info("\n")
logger.info("======================TEST===================")
logger.info("\n\n")
logger.info(f"model save path > {smodel_path}")
start = time.time()
model = SentenceTransformer(smodel_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, 
                                                                  main_similarity=main_similarity, 
                                                                  batch_size=test_batch_size, 
                                                                  name='sts-test', 
                                                                  show_progress_bar=True)
test_evaluator(model, output_path=test_output_path)
logger.info(f"처리시간 > {time.time() - start:.4f}")

In [ ]:
# 마지막 model 저장
#test_output_path = "../../data11/model/sbert/test"
#model.save(test_output_path)